In [1]:
import gym
import pickle
import numpy as np

import os
import glob

In [28]:
env = gym.make('FrozenLake-v0', is_slippery = False)

## Random

In [124]:
total_reward = 0

env.reset()

while 1:
    obser, r, done, info = env.step(env.action_space.sample())

    print("Score: ", obser, r, done, info)
    
    total_reward += r
    window_still_open = env.render()
    if window_still_open == False or done:
        env.close()
        break
    

Score:  1 0.0 False {'prob': 1.0}
  (Right)
SFFF
FHFH
FFFH
HFFG
Score:  0 0.0 False {'prob': 1.0}
  (Left)
SFFF
FHFH
FFFH
HFFG
Score:  0 0.0 False {'prob': 1.0}
  (Up)
SFFF
FHFH
FFFH
HFFG
Score:  0 0.0 False {'prob': 1.0}
  (Left)
SFFF
FHFH
FFFH
HFFG
Score:  0 0.0 False {'prob': 1.0}
  (Left)
SFFF
FHFH
FFFH
HFFG
Score:  0 0.0 False {'prob': 1.0}
  (Left)
SFFF
FHFH
FFFH
HFFG
Score:  4 0.0 False {'prob': 1.0}
  (Down)
SFFF
FHFH
FFFH
HFFG
Score:  5 0.0 True {'prob': 1.0}
  (Right)
SFFF
FHFH
FFFH
HFFG


## Q Learning

In [40]:
q_table = np.zeros((env.observation_space.n, env.action_space.n))

In [41]:
episodes = 1000
alpha = 0.5
gamma = 0.9

In [42]:
results = []

## Training
for _ in range(episodes):
    state = env.reset()
    done = False
    
    results.append(False)
    
    while not done:
        if np.max(q_table[state]) > 0:
            action = np.argmax(q_table[state])
        else:
            action = env.action_space.sample()
        
        next_state, r, done, info = env.step(action)
        
        q_table[state][action] = q_table[state][action] + alpha * (r + gamma*max(q_table[next_state]) - q_table[state][action])
        
        state = next_state
        
        if r:
            results[-1] = True

In [43]:
results.count(True)

936

In [ ]:
## Eval

test_episodes = 100
success = 0

for _ in range(test_episodes):
    state = env.reset()
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        
        state, r, done, info = env.step(action)
        
        success += r
    
print("Success rate: ", success/test_episodes)
        

## Epsilon-Greedy

In [47]:
q_table = np.zeros((env.observation_space.n, env.action_space.n))

In [51]:
episodes = 1000
alpha = 0.5
gamma = 0.9

epsilon = 0.4

In [52]:
results = []

## Training
for _ in range(episodes):
    state = env.reset()
    done = False
    
    results.append(False)
    
    while not done:
        n = np.random.random()
        
        if n < epsilon:
            action = env.action_space.sample()
        else:
            action = np.argmax(q_table[state])
        
        next_state, r, done, info = env.step(action)
        
        q_table[state][action] = q_table[state][action] + alpha * (r + gamma*max(q_table[next_state]) - q_table[state][action])
        
        state = next_state
        
        if r:
            results[-1] = True

In [53]:
results.count(True)

422

In [54]:
q_table

array([[0.531441  , 0.59049   , 0.59049   , 0.531441  ],
       [0.531441  , 0.        , 0.6561    , 0.5640584 ],
       [0.59039443, 0.729     , 0.5535176 , 0.59141841],
       [0.64582714, 0.        , 0.14733417, 0.        ],
       [0.59049   , 0.6561    , 0.        , 0.531441  ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.81      , 0.        , 0.6380125 ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.6561    , 0.        , 0.729     , 0.59049   ],
       [0.6561    , 0.81      , 0.81      , 0.        ],
       [0.729     , 0.9       , 0.        , 0.729     ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.80999842, 0.9       , 0.72757617],
       [0.81      , 0.9       , 1.        , 0.81      ],
       [0.        , 0.        , 0.        , 0.        ]])

In [55]:
## Eval

test_episodes = 100
success = 0

for _ in range(test_episodes):
    state = env.reset()
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        
        state, r, done, info = env.step(action)
        
        success += r
    
print("Success rate: ", success/test_episodes)

Success rate:  1.0


## Deep Q-Learning

In [93]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [132]:
class Net(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 16)        
        self.fc2 = nn.Linear(16, output_dim, bias=False)

    def forward(self, state):
        x = torch.zeros(env.observation_space.n)
        x[state] = 1
        
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        return x

In [158]:
episodes = 1000
lr = 1e-3
gamma = 0.9

epsilon = 0.5

In [159]:
model = Net(env.observation_space.n, env.action_space.n)
optimizer = optim.Adam(model.parameters(), lr=lr)


In [160]:
import random

class Buffer:
   curr_state = []
   next_state = []
   action = []
   reward = []
   done = []
  
   def save(action, curr_state, next_state, reward, done):
       Buffer.action.append(action)
       Buffer.curr_state.append(curr_state)
       Buffer.next_state.append(next_state)
       Buffer.reward.append(reward)
       Buffer.done.append(done)

   def get_sample(num_sample=20):
    #    ids =- random.sample(range(0, len(Buffer.action) - 1), num_sample)
      
       action = []
       curr_state = []
       next_state = []
       reward = []
       done = []
       for i in random.sample(range(0, len(Buffer.action) - 1), num_sample):
           action.append(Buffer.action[i])
           curr_state.append(Buffer.curr_state[i])
           next_state.append(Buffer.next_state[i])
           reward.append(Buffer.reward[i])
           done.append(Buffer.done[i])
          
       return action, curr_state, next_state, reward, done

In [161]:

results = []
## Training
for k in range(episodes):
    state = env.reset()
    done = False
    
    results.append(False)
    
    while not done:
        state_qvals = model(state)
        
        n = np.random.random()
        if n < epsilon:
            action = env.action_space.sample()
        else:
            action = int(torch.argmax(state_qvals))
            
        next_state, r, done, info = env.step(action)
        
        # save current step to buffer
        Buffer.save(action, state, next_state, r, done)
        
        state = next_state
        if r:
            results[-1] = True
        
        
        # train with examples in the buffer
        if 20 < len(Buffer.action):
            buff_action, buff_state, buff_next_state, buff_r, buff_done = Buffer.get_sample()
            
            for i in range(len(buff_action)):
                state_qvals = model(buff_state[i])
                next_state_qvals = model(buff_next_state[i])
                state_q = state_qvals[buff_action[i]]
                real_q = r + gamma*torch.max(next_state_qvals)
            
                loss = F.mse_loss(state_q, real_q)
                loss.backward()
                optimizer.step()
                
    if k % 10 == 0:
        print("Episode:", k, "last loss:", loss)

Episode: 0 last loss: tensor(0., grad_fn=<MseLossBackward0>)
Episode: 10 last loss: tensor(0., grad_fn=<MseLossBackward0>)
Episode: 20 last loss: tensor(0., grad_fn=<MseLossBackward0>)
Episode: 30 last loss: tensor(0., grad_fn=<MseLossBackward0>)
Episode: 40 last loss: tensor(0., grad_fn=<MseLossBackward0>)
Episode: 50 last loss: tensor(0., grad_fn=<MseLossBackward0>)
Episode: 60 last loss: tensor(0., grad_fn=<MseLossBackward0>)
Episode: 70 last loss: tensor(0., grad_fn=<MseLossBackward0>)
Episode: 80 last loss: tensor(0., grad_fn=<MseLossBackward0>)
Episode: 90 last loss: tensor(0., grad_fn=<MseLossBackward0>)
Episode: 100 last loss: tensor(0., grad_fn=<MseLossBackward0>)
Episode: 110 last loss: tensor(0., grad_fn=<MseLossBackward0>)
Episode: 120 last loss: tensor(0., grad_fn=<MseLossBackward0>)
Episode: 130 last loss: tensor(0., grad_fn=<MseLossBackward0>)
Episode: 140 last loss: tensor(0., grad_fn=<MseLossBackward0>)
Episode: 150 last loss: tensor(0., grad_fn=<MseLossBackward0>)
Epi

KeyboardInterrupt: 

In [ ]:
## Eval

test_episodes = 100
success = 0

for _ in range(test_episodes):
    state = env.reset()
    done = False
    
    while not done:
        action = torch.argmax(model(state)) # np.argmax(q_table[state])
        
        state, r, done, info = env.step(action)
        
        success += r
    
print("Success rate: ", success/test_episodes)